In [2]:
import numpy as np

### Question 1
Write a function that receives a parity check matrix $H$ and builds a systematic encoding matrix $G$ for it. This may require altering $H$ by swapping columns, so the function should return two matrices: $\hat{H}$ and $G$, such that $\hat{H}$ is equal to $H$ up to a column permutation and $HGt = 0$ for all $t$ (all the operations are performed in $F2$). Print the outputs of the function for the following matrix:

In [4]:
H = np.array([
    [1, 1, 1, 1, 0, 0],
    [0, 0, 1, 1, 0, 1],
    [1, 0, 0, 1, 1, 0]
], dtype=int)

First, create a function that generates $\hat{H}$ given $H$; this is achieved through Gaussian elimination.

In [5]:
def get_H_hat(H):
    N, K = H.shape[1], H.shape[1] - H.shape[0]
    H_hat = np.copy(H)
    for i in range(0, N - K):
        j = 1
        while H_hat[i, K + i] != 1:
            H_hat[i] = np.mod(H_hat[i] + H_hat[np.mod(i + j, N - K)], 2)
            j += 1
        for j in range(0, N - K):
            if H_hat[j, K + i] == 1 and j != i:
                H_hat[j] = np.mod(H_hat[j] + H_hat[i], 2)
    return H_hat

Then create the required function that given the parity-check matrix $H$ it returns the generator matrix $G$ (along with $\hat{H}$).

In [6]:
def get_identity_matrix(n):
    I = np.zeros((n, n), dtype=int)
    for i in range(n):
        I[i, i] = 1
    return I

def encode(H):
    N, K = H.shape[1], H.shape[1] - H.shape[0]
    H_hat = get_H_hat(H)
    P = np.transpose(H_hat[:,:K])
    I = get_identity_matrix(K)
    G = np.concatenate((I, P), axis=1)
    return H_hat, G

In [7]:
H_hat, G = encode(H_1)

### Question 3
Write an LDPC-decoder based on Loopy Belief Propagation for Binary Symmetric Channel. Specifically, write a function that receives a parity check matrix $\hat{H}$, a received word $y$, a noise ratio $p$ and an optional parameter of a maximum number of iterations (with default value of $20$). The function should return a decoded vector along with the following return code: $0$ for success, $−1$ if the maximum number of iterations is reached without a successful decoding. Try to make your code efficient. Print the result of the decoding for a given parity check matrix $H_1$ (in H1.txt) and vector $y_1$ (in y1.txt). The noise ratio was $p = 0.1$. How many iterations did your algorithm take to converge?

Let us load the supplied parity-check matrix $H_1$ and received message $y_1$.

In [3]:
H_1 = np.loadtxt('H1.txt', dtype=int)
y_1 = np.loadtxt('y1.txt', dtype=int)

Let us create a function that checks the validity of a message against a parity-check matrix.

In [8]:
def is_valid(m, H):
    return np.sum(np.mod(H @ m, 2)) == 0

The following cell contains the initialisation part of the decoding algorithm (as described by D. J. C. MacKay)

In [122]:
def initialise(y, H, p=0.1):
    """ Returns the quantities q^{x}_{mn}, describing the probability
    of bit n of message y being equal to x based on check m.
    
    :param y: the message (signal) as a vector of size (n)
    :param H: the parity-check matrix of size (m, n)
    :param p: the level of noise (between 0 and 1)
    
    :return: a tensor indexed as (m, n, x) for the quantities q """
    Q = np.zeros((*H.shape, 2))
    for m in range(H.shape[0]):
        for n in range(H.shape[1]):
            if H[m, n] == 1:
                for k in [0, 1]:
                    Q[m, n, k] = 1 - p if y[n] == k else p
    return Q

The following cell contains the horizontal and vertical steps, representing the message passing steps between variables and factors.

In [ ]:
def horizontal(H, Q, p=0.1):
    R = np.zeros((*H.shape, 2))
    for m in range(H.shape[0]):
        for n in range(H.shape[1]):
            if H[m, n] == 1:
                delta = 1
                for n_p in range(H.shape[1]):
                    if H[m, n_p] == 1 and n != n_p:
                        delta *= Q[m, n_p, 0] - Q[m, n_p, 1]
                R[m, n, 0] = 1 + delta
                R[m, n, 1] = 1 - delta
    return R


def vertical(H, Q, R, y, p=0.1):
    for m in range(Q.shape[0]):
        for n in range(Q.shape[1]):
            if H[m, n] == 1:
                for k in [0, 1]:
                    Q[m, n, k] = 1 - p if y[n] == k else p
                    for m_p in range(Q.shape[0]):
                        if H[m_p, n] == 1:
                            if m != m_p:
                                Q[m, n, k] *= R[m_p, n, k]
                k = Q[m, n, 0] + Q[m, n, 1]
                Q[m, n, 0] /= k
                Q[m, n, 1] /= k
    return Q  


def estimate(H, Q, R, y, p=0.1):
    x = np.zeros((*y.shape, 2))
    z = np.zeros((*y.shape))
    for n in range(Q.shape[1]):
        for k in [0, 1]:
            x[n, k] = 1 - p if y[n] == k else p
            for m in range(Q.shape[0]):
                if H[m, n] == 1:
                    x[n, k] *= R[m, n, k]
        z[n] = 0 if x[n, 0] > x[n, 1] else 1
    return z


def decode(y, H, maximum_iterations=20, p=0.1):
    Q = initialise(y_1, H_1)
    iterations = 0
    while iterations < maximum_iterations:
        R = horizontal(H, Q)
        Q = vertical(H, Q, R, y)
        x = estimate(H, Q, R, y)
        if is_valid(x, H_hat):
            return x
        iterations += 1
    
    
x = decode(y_1, H_1)

In [123]:
def extract(x):
    message = ""
    for i in range(0, 31):
        character = ""
        for j in range(0, 8):
            character += str(int(x[i * 8 + j]))
        character_number = int(character, 2)
        message += chr(character_number)
    return ''.join(message)

In [124]:
extract(x)

'Happy Holidays! Dmitry&David :)'